In [ ]:
##Author: YSS Narasimha Naidu
## March 15th 2024

import requests
from lxml import etree
from lxml.html import parse
from lxml.html.clean import Cleaner
from lxml.cssselect import CSSSelector
import os


In [ ]:
def page_download(url,inputdir,outputdir):
    base = url.split('/')[-1]
    inputname = inputdir + base + ".html"
    outputname = outputdir + base + ".txt"

    page = requests.get(url)

    open(inputname,"w").write(page.content.decode())


    fp = open(outputname, 'w')
    # cleaner = Cleaner()
    
    #delete_tags= ['span','sup','style','displaystyle','math','small']
    accept_tags= ['p','div']
    doc = parse( inputname).getroot()
    ## get the element with class = "mw-page-container"

    ## get the element in doc that has id body-content


    for element in doc.iter():
        if element.tag in accept_tags:
            for subelem in element:
                if (subelem.tag not in accept_tags):
                    subelem.clear()
                    subelem.text = " "
            fp.write(element.text_content() )




    fp.close()
    

    fp = open( outputname,'r')
    fp.seek(0, os.SEEK_END)
    print("Size of file" + outputname +  " is :", fp.tell(), "bytes")
    fp.close()

In [ ]:
Outputdir = '/home/sasi/Desktop/Final_Sem/CS4805_BTP/Exp4/Sigmetrics/researchxml/'
Inputdir =  '/home/sasi/Desktop/Final_Sem/CS4805_BTP/Exp4/Sigmetrics/researchpapers/'
FinalTextdir = '/home/sasi/Desktop/Final_Sem/CS4805_BTP/Exp4/Sigmetrics/researchtext/'

In [ ]:
from grobid_client.grobid_client import GrobidClient


if __name__ == "__main__":
    client = GrobidClient(config_path="./config.json")
    client.process("processFulltextDocument", Inputdir, output=Outputdir)


In [ ]:

def xmlparser(file,Outputdir,Finaltextdir):
    base = file.split('.')[0]
    inputname =  Outputdir + file
    outputname =  Finaltextdir +"/"+ base+".txt"
    fp = open(outputname, 'w')
    # cleaner = Cleaner()

    delete_tags= ['ref','formula']
    accept_tags= ['p']
    ignore_tags = ['note']
    doc = parse( inputname).getroot()
    ## get the element with class = "mw-page-container"

    ## get the element in doc that has id body-content


    for element in doc.iter():
        if (element.tag in ignore_tags):
            element.clear()
        elif (element.tag in accept_tags) :
            for subelem in element:
                if (subelem.tag in delete_tags):
                    # subelem.clear()
                    subelem.text = " "
            fp.write(element.text_content()  + "\n")

    print(file)


    fp.close()

In [ ]:
##Grobid Command line interface was used to convert the pdf files to xml files.


In [ ]:
import multiprocessing
from joblib import Parallel, delayed

num_cores = multiprocessing.cpu_count()


files = os.listdir(Outputdir)

# for file in files:
#     #chunker(chunk_lst, Articles_dir + file, chunk_len = 32)
#     print(file)
   
#     xmlparser(file,Outputdir,FinalTextdir)

if __name__ == "__main__":
    processed_list = Parallel(n_jobs=num_cores-2)(delayed(xmlparser) (file,Outputdir,FinalTextdir) 
                                                        for file in files  )

    #do gorbet conversion here